# LSA(Latent Semantic Analysis)?


|-|과일이|길고|노란|먹고|바나나|사과|싶은|저는|좋아요|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|문서1|0|0|0|1|0|1|1|0|0|
|문서2|0|0|0|1|1|0|1|0|0|
|문서3|0|1|1|0|2|0|0|0|0|
|문서4|1|0|0|0|0|0|0|1|1|

In [1]:
import numpy as np
# 4 × 9의 크기를 가지는 DTM이 생성
A=np.array([[0,0,0,1,0,1,1,0,0],
            [0,0,0,1,1,0,1,0,0],
            [0,1,1,0,2,0,0,0,0],
            [1,0,0,0,0,0,0,1,1]])
np.shape(A)

(4, 9)

## full SVD 시도

In [11]:
U, s, VT = np.linalg.svd(A, full_matrices = True)

In [5]:
# 4 × 4의 크기를 가지는 직교 행렬 U가 생성
print(U.round(2))
np.shape(U)

[[-0.24  0.75  0.   -0.62]
 [-0.51  0.44 -0.    0.74]
 [-0.83 -0.49 -0.   -0.27]
 [-0.   -0.    1.    0.  ]]


(4, 4)

In [6]:
# 대각 행렬 S를 확인
print(s.round(2))
np.shape(s)

[2.69 2.05 1.73 0.77]


(4,)

In [7]:
# Numpy의 linalg.svd()는 특이값 분해의 결과로 대각 행렬이 아니라, 
# 특이값의 리스트를 반환합니다. -> 이를 다시 대각 행렬로 바꾸어 주어야 합니다.
# 우선 특이값을 s에 저장하고 대각 행렬 크기의 행렬을 생성한 후에,
# 그 행렬에 특이값을 삽입해도록 하겠습니다.

S = np.zeros((4, 9)) # 대각 행렬의 크기인 4 x 9의 임의의 행렬 생성
S[:4, :4] = np.diag(s) # 특이값을 대각행렬에 삽입
print(S.round(2))
np.shape(S)

[[2.69 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   2.05 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   1.73 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.77 0.   0.   0.   0.   0.  ]]


(4, 9)

In [9]:
# 9 × 9의 크기를 가지는 직교 행렬 VT(V의 전치 행렬)가 생성
print(VT.round(2))
np.shape(VT)

[[-0.   -0.31 -0.31 -0.28 -0.8  -0.09 -0.28 -0.   -0.  ]
 [ 0.   -0.24 -0.24  0.58 -0.26  0.37  0.58 -0.   -0.  ]
 [ 0.58 -0.    0.    0.   -0.    0.   -0.    0.58  0.58]
 [ 0.   -0.35 -0.35  0.16  0.25 -0.8   0.16 -0.   -0.  ]
 [-0.   -0.78 -0.01 -0.2   0.4   0.4  -0.2   0.    0.  ]
 [-0.29  0.31 -0.78 -0.24  0.23  0.23  0.01  0.14  0.14]
 [-0.29 -0.1   0.26 -0.59 -0.08 -0.08  0.66  0.14  0.14]
 [-0.5  -0.06  0.15  0.24 -0.05 -0.05 -0.19  0.75 -0.25]
 [-0.5  -0.06  0.15  0.24 -0.05 -0.05 -0.19 -0.25  0.75]]


(9, 9)

In [10]:
#  U × S × VT를 하면 기존의 행렬 A가 나와야 합니다. 
# Numpy의 allclose()는 2개의 행렬이 동일하면 True를 리턴합니다. 
# 기존의 행렬 A와 동일한지 확인해보겠습니다.

np.allclose(A, np.dot(np.dot(U,S), VT).round(2))

True

## 절단된 SVD(Truncated SVD)를 수행

In [12]:
# t=2 -> 우선 대각 행렬 S 내의 특이값 중에서 상위 2개만 남기고 제거
S=S[:2,:2]
print(S.round(2))

[[2.69 0.  ]
 [0.   2.05]]


In [13]:
# 직교 행렬 U에 대해서도 2개의 열만 남기고 제거
U=U[:,:2]
print(U.round(2))

[[-0.24  0.75]
 [-0.51  0.44]
 [-0.83 -0.49]
 [-0.   -0.  ]]


In [15]:
# 행렬 V의 전치 행렬인 VT에 대해서 2개의 행만 남기고 제거
# (V관점에서는 2개의 열만 남기고 제거한 것)
VT=VT[:2,:]
print(VT.round(2))

[[-0.   -0.31 -0.31 -0.28 -0.8  -0.09 -0.28 -0.   -0.  ]
 [ 0.   -0.24 -0.24  0.58 -0.26  0.37  0.58 -0.   -0.  ]]


In [16]:
# 이제 축소된 행렬 U, S, VT에 대해서 다시 U × S × VT연산을 하면, 
# 기존의 A와는 다른 결과가 나오게 됩니다. 값이 손실되었기 때문에,
# 이 세 개의 행렬로는 이제 기존의 A행렬을 복구할 수 없습니다. 
# U × S × VT연산을 해서 나오는 값을 A_prime이라 하고 기존의 행렬 A와 값을 비교
A_prime=np.dot(np.dot(U,S), VT)
print(A)
print(A_prime.round(2))

[[0 0 0 1 0 1 1 0 0]
 [0 0 0 1 1 0 1 0 0]
 [0 1 1 0 2 0 0 0 0]
 [1 0 0 0 0 0 0 1 1]]
[[ 0.   -0.17 -0.17  1.08  0.12  0.62  1.08 -0.   -0.  ]
 [ 0.    0.2   0.2   0.91  0.86  0.45  0.91  0.    0.  ]
 [ 0.    0.93  0.93  0.03  2.05 -0.17  0.03  0.    0.  ]
 [ 0.    0.    0.    0.    0.   -0.    0.    0.    0.  ]]


대체적으로 기존에 0인 값들은 0에 가가운 값이 나오고, 1인 값들은 1에 가까운 값이 나오는 것을 볼 수 있습니다. 또한 값이 제대로 복구되지 않은 구간도 존재해보입니다. 이제 이렇게 차원이 축소된 U, S, VT의 크기가 어떤 의미를 가지고 있는지 알아봅시다.
  
축소된 U는 4 × 2의 크기를 가지는데, 이는 잘 생각해보면 **문서의 개수 × 토픽의 수 t의 크기**입니다. 단어의 개수인 9는 유지되지 않는데 문서의 개수인 4의 크기가 유지되었으니 4개의 문서 각각을 2개의 값으로 표현하고 있습니다. **즉, U의 각 행은 잠재 의미를 표현하기 위한 수치화 된 각각의 문서 벡터**라고 볼 수 있습니다.   
축소된 VT는 2 × 9의 크기를 가지는데, 이는 잘 생각해보면 토픽의 수 **t × 단어의 개수의 크기**입니다. **VT의 각 열은 잠재 의미를 표현하기 위해 수치화된 각각의 단어 벡터**라고 볼 수 있습니다.
  
이 문서 벡터들과 단어 벡터들을 통해 다른 문서의 유사도, 다른 단어의 유사도, 단어(쿼리)로부터 문서의 유사도를 구하는 것들이 가능해집니다.

---
# LSA 실습
사이킷런에서는 Twenty Newsgroups이라고 불리는 20개의 다른 주제를 가진 뉴스그룹 데이터를 제공합니다. 앞서 언급했듯이 LSA가 토픽 모델링에 최적화 된 알고리즘은 아니지만, 토픽 모델링이라는 분야의 시초가 되는 알고리즘입니다. 여기서는 LSA를 사용해서 문서의 수를 원하는 토픽의 수로 압축한 뒤에 각 토픽당 가장 중요한 단어 5개를 출력하는 실습으로 토픽 모델링을 수행합니다.

뉴스그룹 데이터는 뉴스 데이터가 아닙니다.

## 뉴스그룹 데이터에 대한 이해

In [17]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
len(documents)

11314

In [18]:
documents[1]

"\n\n\n\n\n\n\nYeah, do you expect people to read the FAQ, etc. and actually accept hard\natheism?  No, you need a little leap of faith, Jimmy.  Your logic runs out\nof steam!\n\n\n\n\n\n\n\nJim,\n\nSorry I can't pity you, Jim.  And I'm sorry that you have these feelings of\ndenial about the faith you need to get by.  Oh well, just pretend that it will\nall end happily ever after anyway.  Maybe if you start a new newsgroup,\nalt.atheist.hard, you won't be bummin' so much?\n\n\n\n\n\n\nBye-Bye, Big Jim.  Don't forget your Flintstone's Chewables!  :) \n--\nBake Timmons, III"

뉴스그룹 데이터에는 특수문자가 포함된 다수의 영어문장으로 구성되어져 있습니다. 이런 형식의 샘플이 총 11,314개 존재합니다. 사이킷런이 제공하는 뉴스그룹 데이터에서 target_name에는 본래 이 뉴스그룹 데이터가 어떤 20개의 카테고리를 갖고있었는지가 저장되어져 있습니다. 이를 출력해보겠습니다.

In [19]:
print(dataset.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


## 텍스트 전처리

시작하기 앞서, 텍스트 데이터에 대해서 가능한한 정제 과정을 거쳐야만 합니다. 기본적인 아이디어는 알파벳을 제외한 구두점, 숫자, 특수 문자를 제거하는 것입니다. 이는 텍스트 전처리 챕터에서 정제 기법으로 배웠던 정규 표현식을 통해서 해결할 수 있습니다. 또한 짧은 단어는 유용한 정보를 담고있지 않다고 가정하고, 길이가 짧은 단어도 제거합니다. 그리고 마지막으로 모든 알파벳을 소문자로 바꿔서 단어의 개수를 줄이는 작업을 합니다.

In [20]:
news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

In [21]:
#  첫번째 훈련용 뉴스그룹 샘플만 출력하여 정제 전, 후에 어떤 차이가 있는지 확인
news_df['clean_doc'][1]

'yeah expect people read actually accept hard atheism need little leap faith jimmy your logic runs steam sorry pity sorry that have these feelings denial about faith need well just pretend that will happily ever after anyway maybe start newsgroup atheist hard bummin much forget your flintstone chewables bake timmons'

In [22]:
# 뉴스그룹 데이터에서 불용어를 제거 
# 불용어를 제거하기 위해서 토큰화를 우선 수행합니다. 토큰화와 불용어 제거를 순차적으로 진행합니다.
from nltk.corpus import stopwords
stop_words = stopwords.words('english') # NLTK로부터 불용어를 받아옵니다.
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

In [24]:
# 첫번째 훈련용 뉴스그룹 샘플을 출력
print(tokenized_doc[1])
#기존에 있었던 불용어에 속하던 your, about, just, that, will, after 단어들이 사라졌을 뿐만 아니라, 토큰화가 수행됨.

['yeah', 'expect', 'people', 'read', 'actually', 'accept', 'hard', 'atheism', 'need', 'little', 'leap', 'faith', 'jimmy', 'logic', 'runs', 'steam', 'sorry', 'pity', 'sorry', 'feelings', 'denial', 'faith', 'need', 'well', 'pretend', 'happily', 'ever', 'anyway', 'maybe', 'start', 'newsgroup', 'atheist', 'hard', 'bummin', 'much', 'forget', 'flintstone', 'chewables', 'bake', 'timmons']


## TF-IDF 행렬 만들기
불용어 제거를 위해 토큰화 작업을 수행하였지만, TfidfVectorizer(TF-IDF 챕터 참고)는 기본적으로 토큰화가 되어있지 않은 텍스트 데이터를 입력으로 사용합니다. 그렇기 때문에 TfidfVectorizer를 사용해서 TF-IDF 행렬을 만들기 위해서 다시 토큰화 작업을 역으로 취소하는 작업을 수행해보도록 하겠습니다. 이를 **역토큰화(Detokenization)**라고 합니다.

In [25]:
# 역토큰화 (토큰화 작업을 역으로 되돌림)
detokenized_doc = []
for i in range(len(news_df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

news_df['clean_doc'] = detokenized_doc

In [27]:
#정상적으로 불용어가 제거된 상태에서 역토큰화가 수행되었음을 확인
news_df['clean_doc'][1]

'yeah expect people read actually accept hard atheism need little leap faith jimmy logic runs steam sorry pity sorry feelings denial faith need well pretend happily ever anyway maybe start newsgroup atheist hard bummin much forget flintstone chewables bake timmons'

사이킷런의 TfidfVectorizer를 통해 단어 1,000개에 대한 TF-IDF 행렬을 만들 것입니다. 물론 텍스트 데이터에 있는 모든 단어를 가지고 행렬을 만들 수는 있겠지만, 여기서는 1,000개의 단어로 제한하도록 하겠습니다.

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', 
max_features= 1000, # 상위 1,000개의 단어를 보존 
max_df = 0.5, 
smooth_idf=True)

X = vectorizer.fit_transform(news_df['clean_doc'])
X.shape # TF-IDF 행렬의 크기 확인

(11314, 1000)

## LSA -토픽 모델링(Topic Modeling)

이제 TF-IDF 행렬을 다수의 행렬로 분해해보도록 하겠습니다. 여기서는 **사이킷런의 절단된 SVD(Truncated SVD)를 사용**합니다. 절단된 SVD를 사용하면 차원을 축소할 수 있습니다. 원래 기존 뉴스그룹 데이터가 20개의 카테고리를 갖고있었기 때문에, 20개의 토픽을 가졌다고 가정하고 토픽 모델링을 시도해보겠습니다. **토픽의 숫자는 n_components의 파라미터로 지정이 가능**합니다.

In [31]:
from sklearn.decomposition import TruncatedSVD
svd_model = TruncatedSVD(n_components=20, algorithm='randomized', n_iter=100, random_state=122)
svd_model.fit(X)
len(svd_model.components_)

20

In [34]:
# svd_model.componets_는 앞서 배운 LSA에서 VT에 해당
# 토픽의 수 t × 단어의 수의 크기
np.shape(svd_model.components_)

(20, 1000)

In [35]:
terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(svd_model.components_,terms)

Topic 1: [('like', 0.21378), ('know', 0.20032), ('people', 0.19317), ('think', 0.17807), ('good', 0.15105)]
Topic 2: [('thanks', 0.3292), ('windows', 0.29094), ('card', 0.18016), ('drive', 0.1739), ('mail', 0.15126)]
Topic 3: [('game', 0.37161), ('team', 0.32553), ('year', 0.28204), ('games', 0.25416), ('season', 0.18464)]
Topic 4: [('drive', 0.52804), ('scsi', 0.20035), ('disk', 0.15514), ('hard', 0.15506), ('card', 0.14041)]
Topic 5: [('windows', 0.4052), ('file', 0.25609), ('window', 0.18057), ('files', 0.1619), ('program', 0.1401)]
Topic 6: [('chip', 0.16138), ('government', 0.16026), ('mail', 0.15625), ('space', 0.1505), ('information', 0.13575)]
Topic 7: [('like', 0.67109), ('bike', 0.14268), ('know', 0.11428), ('chip', 0.11058), ('sounds', 0.10398)]
Topic 8: [('card', 0.45115), ('sale', 0.21607), ('video', 0.21391), ('monitor', 0.14913), ('offer', 0.14872)]
Topic 9: [('know', 0.44979), ('card', 0.35491), ('chip', 0.17206), ('video', 0.15184), ('government', 0.15053)]
Topic 10: [

# 5. LSA의 장단점(Pros and Cons of LSA)
정리해보면 LSA는 쉽고 빠르게 구현이 가능할 뿐만 아니라 단어의 잠재적인 의미를 이끌어낼 수 있어 문서의 유사도 계산 등에서 좋은 성능을 보여준다는 장점을 갖고 있습니다. 하지만 SVD의 특성상 이미 계산된 LSA에 새로운 데이터를 추가하여 계산하려고하면 보통 처음부터 다시 계산해야 합니다. 즉, 새로운 정보에 대해 업데이트가 어렵습니다. 이는 최근 LSA 대신 Word2Vec 등 단어의 의미를 벡터화할 수 있는 또 다른 방법론인 인공 신경망 기반의 방법론이 각광받는 이유이기도 합니다.

